In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb

from bs4 import BeautifulSoup
import requests
from requests.compat import quote
from PIL import Image, ImageFile
from openpyxl import Workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import sanitize_filename

In [2]:
def get_image_dimensions (uri, headers) :
    image_url = quote(uri.encode('utf-8'), ':/')
    try: 
        r = requests.get(image_url, headers=headers)
        image = Image.open(BytesIO(r.content))
        return image.size[0] * image.size[1]
    except :
        print ('Exception')
        return 0

In [3]:
commons_site = pb.Site("commons", "commons")
wb = Workbook()
ws = wb.active
EXCEL_FILE = "diario.madrid.es.xlsx"

In [4]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [5]:
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
headers = {'User-Agent': user_agent}
url_base = 'https://diario.madrid.es/blog/notas-de-prensa/'

r = requests.get(url_base, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
pages = soup.find_all("a", class_="page-numbers")
final_page = int(pages[-2].get_text())-1
final_page

206

In [6]:
page_number = 3
row = []
for i in range (final_page) :
    if i > page_number :
        break

    if i == 0 :
        url = url_base
    else :
        url = '{0}page/{1}'.format(url_base, i+1)
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    articles = soup.find_all("h2", class_="post-title")
    for article in articles :
        article_url = article.a["href"]
        print('-----')
        print(article_url)
        r = requests.get(article_url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
        print (title)
        date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
        year = date.split('-')[0]
        month = calendar.month_name[int(date.split('-')[1])]
        body = soup.find_all("div", class_="post-content")
        p_description = body[0].find_all("p")
        #print (p_description)
        for p in p_description :
            if len(p.get_text()) > 10:
                description = p.get_text()
                break

        counter = 0
        images = [image.a["href"] for image in soup.find_all("div", class_="gallery-icon")]
        for image in images:
            try:
                image_url = quote(image.encode('utf-8'), ':/')
            except :
                continue
            print (image_url)
            image_name = sanitize_filename('{} {:02d}.jpg'.format(title, counter+1))
            image_path = os.path.join(images_directory, image_name)
            print (image_path)
            try:
                r = requests.get(image_url, headers=headers)
                image = Image.open(BytesIO(r.content))
                image.save(image_path)
                row = [image_path, image_url, title, article_url, date, year, month, description]
                ws.append(row)
                pass
            except :
                print ('Failed download. Skipping')
            counter += 1

        images = soup.find_all(attrs={'class': re.compile(r"wp-image-\d*")})
        for i, image in enumerate(images) :
            try:
                links = {int(x.strip().split(' ')[1][:-1]): x.strip().split(' ')[0] for x in image.get('srcset').split(',')}
                sortedlist = [(k, links[k]) for k in sorted(links)]
                link = sortedlist[-1][1]
            except :
                link = image.get('src')
                
            if image.parent.get('href') == None :
                # No parent
                print ('No parent')
                image_url = link
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            elif get_image_dimensions(image.parent.get('href'), headers) > get_image_dimensions(link, headers) :
                # Bigger size
                print ('Bigger size')
                image_url = image.parent.get('href')
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            else :
                # Others
                print ('Other')
                image_url = link
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            image_name = sanitize_filename('{} {:02d}.jpg'.format(title, counter+1))
            image_path = os.path.join(images_directory, image_name)
            print (image_path)

            #print (image_path)
            try:
                r = requests.get(image_url, headers=headers)
                image = Image.open(BytesIO(r.content))
                image.save(image_path)
                row = [image_path, image_url, title, article_url, date, year, month, description]
                ws.append(row)
                pass
            except :
                print ('Failed download. Skipping')
            counter += 1

-----
https://diario.madrid.es/blog/notas-de-prensa/el-pleno-aprueba-la-creacion-de-una-comision-de-investigacion-de-bicimad/
El Pleno aprueba la creación de una comisión de investigación de BiciMad
-----
https://diario.madrid.es/blog/notas-de-prensa/naves-matadero-estrena-mystery-magnet-de-la-artista-belga-miet-warlop/
Naves Matadero estrena Mystery Magnet, de la artista belga Miet Warlop
Other
https://diario.madrid.es/wp-content/uploads/2018/01/DSC_3747%E2%88%8F20Jose20Caldeira_preview.jpeg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Naves Matadero estrena Mystery Magnet, de la artista belga Miet Warlop 01.jpg
-----
https://diario.madrid.es/blog/notas-de-prensa/la-afiliacion-a-la-seguridad-social-crece-un-41-interanual-en-la-ciudad-de-madrid/
La afiliación a la Seguridad Social crece un 4,1% interanual en la ciudad de Madrid
-----
https://diario.madrid.es/blog/notas-de-prensa/madrid-se-suma-a-la-declaracion-de-sevilla-y-se-compromete-con-la-economia-circular/
Madr

https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaEsculturaPedroDuque_4-1500x1000.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Ciudad Lineal rinde homenaje a su vecino más internacional, el astronauta Pedro Duque 03.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaEsculturaPedroDuque_1-1413x1000.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Ciudad Lineal rinde homenaje a su vecino más internacional, el astronauta Pedro Duque 04.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaEsculturaPedroDuque_5-1500x1000.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Ciudad Lineal rinde homenaje a su vecino más internacional, el astronauta Pedro Duque 05.jpg
https://diario.madrid.es/wp-content/uploads/2018/01/Gal_ManuelaCarmenaEsculturaPedroDuque_6-1500x1000.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Ciudad Lineal rinde homenaje a su vecino 

-----
https://diario.madrid.es/blog/notas-de-prensa/428-millones-usuarios-eligieron-emt-en-2017/
428 millones usuarios eligieron EMT en 2017
https://diario.madrid.es/wp-content/uploads/2018/01/Hibridos_viajeros_14-1494x1000.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\428 millones usuarios eligieron EMT en 2017 01.jpg
-----
https://diario.madrid.es/blog/notas-de-prensa/seleccionados-5-finalistas-de-los-proyectos-para-viviendas-intergeneracionales-para-mayores-y-jovenes-en-vallecas/
Cinco finalistas de los proyectos de viviendas para mayores y jóvenes en Vallecas
Bigger size
https://diario.madrid.es/wp-content/uploads/2018/01/sede-EMVS.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Cinco finalistas de los proyectos de viviendas para mayores y jóvenes en Vallecas 01.jpg
-----
https://diario.madrid.es/blog/notas-de-prensa/mercado-de-productores-apuesta-por-el-consumo-responsable-en-matadero-madrid/
Mercado de Productores apuesta por el consumo r

-----
https://diario.madrid.es/blog/notas-de-prensa/oposiciones-para-aumentar-la-plantilla-de-policia-municipal-en-99-plazas/
Oposiciones para aumentar la plantilla de Policía Municipal en 99 plazas
Other
https://diario.madrid.es/wp-content/uploads/2018/01/D_17-044_062.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Oposiciones para aumentar la plantilla de Policía Municipal en 99 plazas 01.jpg
Other
https://diario.madrid.es/wp-content/uploads/2018/01/D_17-037-Secci%C2%A6n-Canina-Uniformidad-2017-JPG-90.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Oposiciones para aumentar la plantilla de Policía Municipal en 99 plazas 02.jpg
-----
https://diario.madrid.es/blog/notas-de-prensa/aprobado-el-proyecto-de-integracion-del-estadio-santiago-bernabeu-en-su-entorno-urbano/
Aprobado el proyecto de integración del estadio Santiago Bernabéu en su entorno urbano
Bigger size
https://diario.madrid.es/wp-content/uploads/2018/01/Gal_RuedaPrensaJMCalvo_1.jpg
D:\

In [7]:
wb.save(os.path.join(cwd, EXCEL_FILE))